In [1]:
from aocd import get_data

In [3]:
import numpy as np

# Day 9: Disk Fragmenter

## Part One

In [153]:
example_input = "2333133121414131402"

In [154]:
counts = np.array(list(example_input), dtype=int); counts

array([2, 3, 3, 3, 1, 3, 3, 1, 2, 1, 4, 1, 4, 1, 3, 1, 4, 0, 2])

In [155]:
len(counts)

19

In [156]:
file_values = np.arange(0, len(list(example_input))//2 + 1)
empty_values = np.full(len(list(example_input))//2 + 1, '.')

In [157]:
interleaved = list(itertools.chain(*zip(file_values, empty_values)))

In [158]:
blocks = np.repeat(interleaved[:len(counts)], counts); blocks

array(['0', '0', '.', '.', '.', '1', '1', '1', '.', '.', '.', '2', '.',
       '.', '.', '3', '3', '3', '.', '4', '4', '.', '5', '5', '5', '5',
       '.', '6', '6', '6', '6', '.', '7', '7', '7', '.', '8', '8', '8',
       '8', '9', '9'], dtype='<U21')

In [159]:
''.join(blocks)

'00...111...2...333.44.5555.6666.777.888899'

In [82]:
def shift_blocks(arr):
    while '.' in arr:
        value = arr[-1]  # get last element 
        arr = np.delete(arr, -1)  # remove last element, numpy doesn't have pop()
        
        dot_index = np.where(arr == '.')[0][0]
        arr[dot_index] = value
    
    return arr
        

In [83]:
blocks = shift_blocks(blocks)

In [84]:
blocks

array(['0', '0', '9', '9', '8', '1', '1', '1', '8', '8', '8', '2', '7',
       '7', '7', '3', '3', '3', '6', '4', '4', '6', '5', '5', '5', '5',
       '6', '6'], dtype='<U21')

In [17]:
positions = np.arange(0, len(blocks))
checksum = np.sum(positions * blocks.astype(int))

In [18]:
checksum

np.int64(1928)

`shift_blocks` isn't efficient for a larger dataset - we can do better

In [149]:
def replace_dots_with_reversed(arr):
    # reverse array and remove dots
    reversed_array = arr[::-1][arr[::-1] != '.']
    # find all indices for dots and replace
    dot_indices = np.where(arr == '.')
    arr[dot_indices] = reversed_array[:len(arr[dot_indices])]
    # num of ints in array should be the same as originally
    return arr[:len(reversed_array)]

In [150]:
test_array = np.array(['1','.','.','2','.','3','4'])

In [152]:
replace_dots_with_reversed(test_array)

array(['1', '4', '3', '2'], dtype='<U1')

In [160]:
replace_dots_with_reversed(blocks)

array(['0', '0', '9', '9', '8', '1', '1', '1', '8', '8', '8', '2', '7',
       '7', '7', '3', '3', '3', '6', '4', '4', '6', '5', '5', '5', '5',
       '6', '6'], dtype='<U21')

In [174]:
data = get_data(day=9, year=2024)

In [161]:
import itertools

def calculate(data):
    # set up data
    counts = np.array(list(data), dtype=int)
    file_values = np.arange(0, len(data)//2 + 1)
    empty_values = np.full(len(data)//2 + 1, '.')
    interleaved = list(itertools.chain(*zip(file_values, empty_values)))
    blocks = np.repeat(interleaved[:len(counts)], counts)

    # run "defragger"
    blocks = replace_dots_with_reversed(blocks)
    
    positions = np.arange(0, len(blocks))
    checksum = np.sum(positions * blocks.astype(int))
    
    return checksum

In [163]:
calculate(data)

np.int64(6370402949053)

## Part Two

The eager amphipod already has a new plan: rather than move individual blocks, he'd like to try compacting the files on his disk by moving whole files instead.

This time, attempt to move whole files to the leftmost span of free space blocks that could fit the file. Attempt to move each file exactly once in order of decreasing file ID number starting with the file with the highest file ID number. If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.

Start over, now compacting the amphipod's hard drive using this new method instead. What is the resulting filesystem checksum?



In [223]:
keys = np.array(list(data), dtype=int)

file_values = np.arange(0, len(data)//2 + 1)
empty_values = np.full(len(data)//2 + 1, '.')
values = list(itertools.chain(*zip(file_values, empty_values)))[:len(keys)]

counts = np.stack((keys, values), axis=1)

In [224]:
counts

array([['1', '0'],
       ['2', '.'],
       ['3', '1'],
       ...,
       ['8', '9998'],
       ['9', '.'],
       ['7', '9999']], dtype='<U21')

In [225]:
r_keys = np.array(list(reversed(keys)))
r_values = np.array(list(reversed(values)))

In [244]:
reversed_counts = counts[::-1][(reversed_counts[:, 1] != '.')]

In [245]:
reversed_counts

array([['7', '9999'],
       ['8', '9998'],
       ['7', '9997'],
       ...,
       ['6', '2'],
       ['3', '1'],
       ['1', '0']], dtype='<U21')

In [210]:
indices = np.where(np.array(values[:len(keys)]) == '.')
counts[indices]

array([['2', '.'],
       ['4', '.'],
       ['4', '.'],
       ...,
       ['0', '.'],
       ['7', '.'],
       ['9', '.']], dtype='<U21')

In [230]:
condition = (reversed_counts[:, 0] == '7') & (reversed_counts[:, 1] != '.')
reversed_counts[condition]

array([['7', '9999'],
       ['7', '9997'],
       ['7', '9990'],
       ...,
       ['7', '73'],
       ['7', '31'],
       ['7', '14']], dtype='<U21')

In [246]:
for i in range(1, 10):
    print(i)
    replace_condition = (counts[:, 0] == f'{i}') & (counts[:, 1] == '.')    
    print(len(counts[replace_condition]))
    condition = (reversed_counts[:, 0] == f'{i}') & (reversed_counts[:, 1] != '.')
    print(len(reversed_counts[condition]))

    counts[replace_condition][:len(reversed_counts[condition])] = reversed_counts[condition][:len(counts[replace_condition])]

1
0
1146
2
1070
1059
3
958
1119
4
1014
1117
5
979
1058
6
1015
1163
7
998
1105
8
1035
1106
9
1035
1127


Need to extend 